# Simulate CAMISIM and MeSS reads

## Clone repo

In [ ]:
git clone https://github.com/metagenlab/benchmark-MeSS-CAMISIM

## Setup NCBI API key

Go to https://www.ncbi.nlm.nih.gov/account/login/ to create an NCBI account and get an API key.

Before launching the pipeline set your key as a secret. 

Example:

In [ ]:
nextflow secrets set NCBI_KEY 0123456789abcdef

## Get Diversities data

### Get simulated reads 

In [ ]:
container_prefix=$(realpath ./containers)
nextflow run benchmark-MeSS-CAMISIM/main.nf \\
        -params-file benchmark-MeSS-CAMISIM/run-params.yml \\
        -c benchmark-MeSS-CAMISIM/nextflow.config \\
        --input data/samples \\
        --outdir reads/simulated \\
        --container_prefix $container_prefix  

### Symlink simulated reads

In [ ]:
mkdir sim_fastq
ln -s reads/simulated/camisim/camisim_SRS* sim_fastq
ln -s reads/simulated/mess/*/*.fq.gz sim_fastq

### Classify simulated reads

In [ ]:
db_path=$(realpath ./k2db)
nextflow run classify.nf -c nextflow.config \\
        --input sim_fastq \\
        --output reads/simulated/classified \\
        --archive false \\
        --fastp true \\
        --threshold 200 \\ 
        --db $db_path 

> [!TIP]  
> abundances from data/abundances are in reads/simulated/classified/abundances

### Get otu table 

In [ ]:
./get_tables.py reads/simulated/classified/abundances

### Get lineage table 

In [ ]:
taxonkit lineage data/uniq_taxids.tsv | \\
taxonkit reformat | csvtk -H -t cut -f 1,3 | \\
csvtk -H -t sep -f 2 -s ';' -R | \\
csvtk add-header -t -n taxid,kindom,phylum,class,order,family,genus,species > data/lineage.tsv

## Get resource usage data

In [ ]:
container_prefix=$(realpath ./containers)
nextflow run benchmark-MeSS-CAMISIM/main.nf \\
        -params-file benchmark-MeSS-CAMISIM/subsets-params.yml \\
        -c benchmark-MeSS-CAMISIM/nextflow.config \\
        --outdir benchmark \\
        --container_prefix $container_prefix  

> [!TIP]  
> Resources usage in benchmark/pipeline_info/execution_trace_*.txt